In [6]:
import pprint
import os
import torch
import transformers
import yaml
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv

load_dotenv()

True

# Load Models

## Load Embedding Model

In [2]:
model_name = 'sentence-transformers/sentence-t5-base'
model_kwargs = {'device': 'cuda'}

hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,

)

## Load Llama 2 Model

In [17]:
config_path =  '../configs/model/Llama_2_7b.yaml'

# Load configuration from the YAML file
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

model_id = config['model']['id']
device = config['model']['device']

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=config['bits_and_bytes']['load_in_4bit'],
    bnb_4bit_quant_type=config['bits_and_bytes']['bnb_4bit_quant_type'],
    bnb_4bit_use_double_quant=config['bits_and_bytes']['bnb_4bit_use_double_quant'],
    bnb_4bit_compute_dtype=getattr(torch, config['bits_and_bytes']['bnb_4bit_compute_dtype'])
)

hf_auth = os.getenv("HF_TOKEN")

model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map=device if device != 'auto' else 'auto',
    use_auth_token=hf_auth
)

model.eval()

actual_device = f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu'
print(f"Model loaded on {actual_device}")


/opt/conda/envs/NLP/lib/python3.9/site-packages/transformers/models/auto/configuration_auto.py:1033: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/opt/conda/envs/NLP/lib/python3.9/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/envs/NLP/lib/python3.9/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Model loaded on cuda:0


In [19]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

stop_list = ['\nHuman:', '\n```\n']

stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
stop_token_ids

import torch

stop_token_ids = [torch.LongTensor(x).to('cuda') for x in stop_token_ids]
stop_token_ids

[tensor([    1, 29871,    13, 29950,  7889, 29901], device='cuda:0'),
 tensor([    1, 29871,    13, 28956,    13], device='cuda:0')]

In [20]:
from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [21]:
generate_text = transformers.pipeline(
    model=model, 
    tokenizer=tokenizer,
    return_full_text=True, 
    task='text-generation',
    stopping_criteria=stopping_criteria, 
    temperature=0.1, 
    max_new_tokens=512,
    repetition_penalty=1.1 )

In [22]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [23]:
out = llm(prompt="Complete the following text: \nThe quick brown fox ...",)

In [26]:
print(out)



A) jumped over a lazy dog.
B) ran across the green field.
C) climbed up the tall tree.
D) swam in the blue ocean.

Answer: A) jumped over a lazy dog.


# Load pfizer data and create Vector store

In [31]:
import glob
from langchain.document_loaders import TextLoader

In [38]:
year = 2023
month = 10 

files = glob.glob(f'../data/pfizer/{year}_{month}/*/*.txt')

In [40]:
files

['../data/pfizer/2023_10/announcements/power-partnership-our-mission-better-connect-patient-health-journey.txt',
 '../data/pfizer/2023_10/announcements/easing-healthcare-burden-seasonal-triple-demic-and-pfizers-unwavering-commitment.txt',
 '../data/pfizer/2023_10/announcements/nobel-prize-awarded-scientists-responsible-mrna-breakthrough.txt',
 '../data/pfizer/2023_10/announcements/pfizers-covid-19-oral-antiviral-treatment-transitioning-government-distribution.txt',
 '../data/pfizer/2023_10/announcements/september-social-media-round-dei-cmo-and-rocky-mount-resumes.txt',
 '../data/pfizer/2023_10/announcements/accelerating-equity-takes-spotlight-pfizers-global-dei-summit.txt',
 '../data/pfizer/2023_10/press-releases/pfizer-and-biontech-announce-positive-topline-data-mrna.txt',
 '../data/pfizer/2023_10/press-releases/pfizer-amends-us-government-paxlovid-supply-agreement-and.txt',
 '../data/pfizer/2023_10/press-releases/pfizer-presents-new-data-idweek-2023-highlighting-advances.txt',
 '../d

In [53]:
documents = []
for file in files:
    loader = TextLoader(file)
    documents.append(loader.load()[0])

In [55]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [56]:
vectorstore = FAISS.from_documents(all_splits, hf)

In [58]:
vectorstore.similarity_search("What is the status of RSV vaccine")

[Document(page_content='View the full Prescribing Information. There may be a delay as the document is updated with the latest information. \xa0It will be available as soon as possible. Please check back for the updated full information shortly.\xa0\nU.S. Indication for TRUMENBA® (meningococcal group B vaccine)\nTRUMENBA® (meningococcal group B vaccine) is indicated for active immunization to prevent invasive disease caused by Neisseria meningitidis serogroup B in individuals aged 10 through 25 years of age..Important Safety Information', metadata={'source': '../data/pfizer/2023_10/press-releases/fda-approves-penbrayatm-first-and-only-vaccine-prevention.txt'}),
 Document(page_content='Virus Vaccine), the company’s bivalent respiratory syncytial virus prefusion F (RSVpreF) vaccine for maternal immunization to protect infants against RSV illness.', metadata={'source': '../data/pfizer/2023_10/press-releases/pfizer-presents-new-data-idweek-2023-highlighting-advances.txt'}),
 Document(page_

In [59]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [69]:
#prompt = PromptTemplate.from_template(
#    "Summarize the main themes in these retrieved docs: {docs}"
#)
prompt = PromptTemplate.from_template(
    "Answer the question based on these retrieved docs: {docs}"
)


llm_chain = LLMChain(llm=llm, prompt=prompt)

In [72]:
question = "What pfizer says about the seasonality of COVID-19 ?"
docs = vectorstore.similarity_search(question,k=1)
result = llm_chain(docs)

In [74]:
pprint.pprint(result["text"])

('  Seasonality of COVID-19: The documents highlight the seasonal pattern of '
 'COVID-19, with peaks in fall and winter, similar to other respiratory '
 'diseases. This suggests that co-infections or consecutive respiratory '
 'infections during this period can increase the risk of severe illness. '
 "[Document(page_content='“The COVID-19 pandemic has accelerated the "
 'development and approval of vaccines, with multiple vaccine candidates '
 'already approved for emergency use and more in various stages of clinical '
 'trials. However, the long-term efficacy and safety of these vaccines remain '
 'uncertain, particularly in certain vulnerable populations such as children, '
 'pregnant women, and immunocompromised individuals,” said Dr. Paul Offit, '
 'Director of the Vaccine Education Center at Children’s Hospital of '
 'Philadelphia. “While combination vaccines may offer advantages over '
 'single-disease vaccines, their development and testing must be carefully '
 'considered to e